# Usando las gráficas... 

<img src="Mexico_City_metro.png"> 

Incluimos el paquete donde tenemos la información de todos los nombres de las estaciones.

In [16]:
include("lineas_metro.txt");

GetStationName (generic function with 2 methods)

In [17]:
using LightGraphs
using GraphLayout

In [15]:
#Condiciones iniciales
Num_station = Length_Estaciones()
length_of_lines = Length_Lineas() 

195

Creamos la función tipo `Graph` **MetroNetwork**. <br>
Una **red** será representada por  **vértices** y se conectará a través de **artistas** o **edges**. La notación para una gráfica es <br>
`Graph(V,E) = {V::vertices,E::edges}` 

In [5]:
MetroNetwork(total_lines=Todas_Lineas) = Graph(Length_Estaciones(total_lines))
typeof(MetroNetwork())

Graph (constructor with 7 methods)

Conectamos por aristas las estaciones de cada línea de metro de manera *independiente*. Es decir, sin unir los trasbordos.

In [6]:
# Dado el arreglo de líneas de metro, construimos una red
# y conectamos sus vértices [estación a estación]
# sin tomar en cuenta los trasbordos

function ConnectSimpleNetwork(total_lines=Todas_Lineas)
    metro_network = MetroNetwork(total_lines)
    lines_length  = Length_Lineas(total_lines) 
    
    s = 1
    for i = 1:length(lines_length)
        for station in s:lines_length[i] + s - 2
            add_edge!(metro_network, station, station + 1)
            s += 1
        end
        s += 1
    end
    return metro_network
end

ConnectSimpleNetwork (generic function with 2 methods)

In [7]:
#Obs: 195 - 183 = 12 [líneas independientes de la red]
metro_network_simple = ConnectSimpleNetwork()

{195, 183} undirected graph

Usando la función `GetStationNumber`, y `GetStationName`, hacemos una función que encuentre los puntos de `metro_network_simple` cuyos nombres sean iguales y los una con una arista.
Esta arista representa los trasbordos de una estación a otra.

In [8]:
function ConnectLines(total_lines=Todas_Lineas)
    num_est_tot   = Length_Estaciones(total_lines)
    lines_length  = Length_Lineas(total_lines)
    #metro_network = ConnectSimpleNetwork(total_lines)
    metro_network = MetroNetwork(total_lines)
    for i = 1:num_est_tot
        for j = 1:num_est_tot
            if i == j 
                continue 
                elseif GetStationName(i, total_lines) == GetStationName(j, total_lines)
                if has_edge(metro_network, i, j)
                else
                    add_edge!(metro_network, i, j)
                end
            end
        end
    end
    return metro_network
end

ConnectLines (generic function with 2 methods)

In [9]:
#Obs: 220 - 183 = 37 transbordos en toda la red.
metro_network_transfer = ConnectLines()

{195, 37} undirected graph

Ahora, se le dará un **peso** a las aristas, que cumplirá la función de tiempo de espera, distinguiéndolas en dos tipos: <br> 
 * Arista tipo trasbordo, que denominaremos `transit_edge` <br>
 * Arista tipo conexión normal, que denominaremos `normal_edge`. 


In [11]:
function ConnectFullNetwork_Weighted(normal_edge=1, transit_edge=1.1,total_lines=Todas_Lineas)
    num_est_tot   = Length_Estaciones(total_lines)
    lines_length  = Length_Lineas(total_lines)
    metro_network = MetroNetwork(total_lines)
    edge_dists = zeros(num_est_tot,num_est_tot) #Matriz que guarda las distancias entre aristas
    #ConnectSimpleNetwork, "pesando" los ejes
    s = 1
    for i = 1:length(lines_length)
        for station in s:lines_length[i] + s - 2
            add_edge!(metro_network, station, station + 1)
            edge_dists[station, station + 1] = normal_edge
            edge_dists[station + 1, station] = normal_edge
            s += 1
        end
        s += 1
    end
    #ConnectLines, pesando los trasbordos 
    for i = 1:num_est_tot
        for j = 1:num_est_tot
            if GetStationName(i, total_lines) == GetStationName(j, total_lines)
                if i != j
                    if has_edge(metro_network, i, j)
                    else
                        add_edge!(metro_network, i, j)
                        edge_dists[i, j] = transit_edge
                        edge_dists[j, i] = transit_edge
                    end
                end
            end
        end
    end
    return metro_network, edge_dists
end

ConnectFullNetwork_Weighted (generic function with 4 methods)

In [18]:
normal_edge_dist = 1
transit_edge_dist = 1.1

1.1

In [19]:
MetroNetwork_Weighted, Edge_Dist = ConnectFullNetwork_Weighted(normal_edge_dist, transit_edge_dist);

Ahora hacemos una función que nos regresa la matriz de distancia entre dos estaciones

In [21]:
ConnectFullNetwork_Weighted(1,1.1,Todas_Lineas);

In [22]:
function MatrixDist(transfer_list, total_lines=Todas_Lineas) #transfer list es la lista de trasbordos
    
    MetroNetwork_Weighted, Edge_Dist = ConnectFullNetwork_Weighted(1,1.1,total_lines)
    
    N = Length_Estaciones()
    D = dijkstra_shortest_paths(MetroNetwork_Weighted, 1, edge_dists= Edge_Dist).dists;
    for i = 2:N
        d = dijkstra_shortest_paths(MetroNetwork_Weighted, i, edge_dists= Edge_Dist).dists
        D = hcat(D, d)
    end

    for i = 1:N
        for t1 in transfer_list
            for t2 in transfer_list
                if GetStationName(t1) == GetStationName(t2)
                    
                    if  D[i,t2] < D[i,t1]
                        D[i,t1] = D[i,t2]
                    if  D[t2,i] < D[t1,i]
                        D[t1,i] = D[t2,i]
                    end
                    end
                end
            end
        end
    end
    return D
end

MatrixDist (generic function with 2 methods)

In [23]:
transfer_list = [2,109,133]

3-element Array{Int64,1}:
   2
 109
 133

In [25]:
@time MatrixDist(transfer_list)
D =  MatrixDist(transfer_list)

elapsed time: 0.100386477 seconds (59302800 bytes allocated, 70.46% gc time)


195x195 Array{Float64,2}:
  0.0   1.0   2.0   3.0   4.0   5.0  …  20.2  21.2  22.2  23.2  24.2  25.2
  1.0   0.0   1.0   2.0   3.0   4.0     18.1  19.1  20.1  21.1  22.1  23.1
  2.0   1.0   0.0   1.0   2.0   3.0     20.2  21.2  22.2  23.2  24.2  25.2
  3.0   2.0   1.0   0.0   1.0   2.0     21.2  22.2  23.2  24.2  25.2  26.2
  4.0   3.0   2.0   1.0   0.0   1.0     22.2  23.2  24.2  25.2  26.2  27.2
  5.0   4.0   3.0   2.0   1.0   0.0  …  22.2  23.2  24.2  25.2  26.2  27.2
  6.0   5.0   4.0   3.0   2.0   1.0     21.2  22.2  23.2  24.2  25.2  26.2
  7.0   6.0   5.0   4.0   3.0   2.0     20.2  21.2  22.2  23.2  24.2  25.2
  8.0   7.0   6.0   5.0   4.0   3.0     19.2  20.2  21.2  22.2  23.2  24.2
  9.0   8.0   7.0   6.0   5.0   4.0     19.2  20.2  21.2  22.2  23.2  24.2
 10.0   9.0   8.0   7.0   6.0   5.0  …  18.2  19.2  20.2  21.2  22.2  23.2
 11.0  10.0   9.0   8.0   7.0   6.0     19.2  20.2  21.2  22.2  23.2  24.2
 12.0  10.2  10.0   9.0   8.0   7.0     18.3  19.3  20.3  21.3  22.3  23.3

In [42]:
D[109,3]

1.0

#Monte Carlo

Ya que obtengamos el número de estaciones normales recorridas y los trasbordos. Lo que queremos es lo siguiente: <br>
* Tomamos aletoriamente dos estaciones $inicio = rand(0, N)$ $ fin = rand(0, N)$, pedimos $inicio != fin$ con un if.
* Calculamos el número de estaciones normale y el número de trasbordos y después hacemos lo siguiente
* Cuando un pasajero se sube al metro espera un tiempo $t_{espera}$ a que pase el metro, suponemos que máximo el metro tarda en pasar $t_{max}$ (ej. a lo más tarda 15 minutos, entonces $t_{max} = 15\times60 s$). Suponemos que la distribución es uniforme (aunque después sería bueno asumir distribución gaussiana o poisson... pero después vemos eso).
Entonces $t_{espera} = rand()*t_{max}$. <br>
* Cuando el metro se detiene en una estación se detiene a lo menos $t_{parada_min}$ y a lo más $t_{parada_max}$. Entonces por cada estación que pare el metro se debe de tomar en cuenta $t_{parada} = rand(t_{parada_min}, t_{parada_max})$ (o como sea que se ponga en la función) <br>
* Cuando el pasajero hace un trasbordo entonces tarda en caminar un tiempo $t_{trasbordo}$ y además debe de esperar a que vuelva a pasar el camión. Volvemos a calcular un $t_{espera} = rand()*t_{max}$.
* Finalmente sumamos todo el tiempo y vemos total de estaciones que recorrio y lo guardamos en una lista apropiada. Después vamos con el siguiente pasajero y hacemos lo mismo.

Definimos los tiempos de espera, parada y trasbordo

In [26]:
tprom_espera = 4*60.
tprom_viaje = 1*60.
tprom_parada = 15.
tprom_tras = 2*60.

120.0

In [46]:
function NormalesyTrasbordos(D, transit_edge_dist, normal_edge_dist, i, j)
    num_tras = round(D[i, j]%1, 1)*10
    num_norm = round((D[i, j] - transit_edge_dist*num_tras)/normal_edge_dist, 1)
    return num_norm, num_tras
end

NormalesyTrasbordos (generic function with 1 method)

In [49]:
NormalesyTrasbordos(D,transit_edge_dist, normal_edge_dist, 1, 21)

(8.0,2.0)

In [ ]:
function PasajerosMonteCarlo(DistMatrix, tprom_espera, tprom_viaje, tprom_parada, tprom_tras, N_simulaciones)
    N = length(DistMatrix)
    initial_station = rand(1:N)
    end_station = rand(1:N)
    
    
end